In [60]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
from nltk.corpus import  stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [61]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [62]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [63]:
df=pd.read_csv('train.csv',index_col='id')
tf=pd.read_csv('test.csv',index_col='id')
df

,title,author,text,label
id,,,,
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...
20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0


In [64]:
df.isnull().sum()

title      558
author    1957
text        39
label        0
dtype: int64

In [65]:
df=df.fillna('')
tf=tf.fillna('')

In [66]:
df['content']=df['author']+' '+df['title']
tf['content']=tf['author']+' '+tf['title']

In [67]:
df.drop(['author','title'],axis=1,inplace=True)
df.head()
tf.drop(['author','title'],axis=1,inplace=True)
tf.head()

,text,content
id,,
20800,"PALO ALTO, Calif. — After years of scorning...",David Streitfeld Specter of Trump Loosens Tong...
20801,Russian warships ready to strike terrorists ne...,Russian warships ready to strike terrorists n...
20802,Videos #NoDAPL: Native American Leaders Vow to...,Common Dreams #NoDAPL: Native American Leaders...
20803,"If at first you don’t succeed, try a different...",Daniel Victor Tim Tebow Will Attempt Another C...
20804,42 mins ago 1 Views 0 Comments 0 Likes 'For th...,Truth Broadcast Network Keiser Report: Meme Wa...


In [68]:
X=df.drop('label',axis=1)
y=df['label']

In [69]:
portStem=PorterStemmer()


In [70]:
def stemming(content):
  stemmedContent=re.sub(r'[^a-zA-Z\s]', '', content)
  stemmedContent=stemmedContent.lower()
  stemmedContent=stemmedContent.split()
  stemmedContent=[portStem.stem(word) for word in stemmedContent if not word in stopwords.words('english')]
  stemmedContent=' '.join(stemmedContent)
  return stemmedContent

In [71]:
df.content=df.content.apply(stemming)
df
tf.content=tf.content.apply(stemming)
tf

,text,content
id,,
20800,"PALO ALTO, Calif. — After years of scorning...",david streitfeld specter trump loosen tongu pu...
20801,Russian warships ready to strike terrorists ne...,russian warship readi strike terrorist near al...
20802,Videos #NoDAPL: Native American Leaders Vow to...,common dream nodapl nativ american leader vow ...
20803,"If at first you don’t succeed, try a different...",daniel victor tim tebow attempt anoth comeback...
20804,42 mins ago 1 Views 0 Comments 0 Likes 'For th...,truth broadcast network keiser report meme war e
...,...,...
25995,Of all the dysfunctions that plague the world’...,jodi rosen bangladeshi traffic jam never end n...
25996,WASHINGTON — Gov. John Kasich of Ohio on Tu...,sheryl gay stolberg john kasich sign one abort...
25997,Good morning. (Want to get California Today by...,mike mcphate california today exactli sushi ne...


In [72]:
x=df.content.values
y=df.label.values
x1=tf.content.values


In [73]:
x

array(['darrel lucu hous dem aid didnt even see comey letter jason chaffetz tweet',
       'daniel j flynn flynn hillari clinton big woman campu breitbart',
       'consortiumnewscom truth might get fire', ...,
       'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time',
       'alex ansari nato russia hold parallel exercis balkan',
       'david swanson keep f aliv'], dtype=object)

converting textural data into numerical


In [74]:
vectorizer = TfidfVectorizer()
x=vectorizer.fit_transform(x)
x1=vectorizer.transform(x1)

In [75]:
print(x)

  (0, 16971)	0.2744592444374437
  (0, 2718)	0.35193713958172035
  (0, 8377)	0.23725805939064787
  (0, 9359)	0.2796386126995312
  (0, 3220)	0.23629392780243347
  (0, 14585)	0.24777447435872538
  (0, 5368)	0.22334412146584304
  (0, 4332)	0.2840070081977437
  (0, 282)	0.25980951706150995
  (0, 4108)	0.25896963868992834
  (0, 7647)	0.21013697798700734
  (0, 9667)	0.34805490263423844
  (0, 3899)	0.3445106979827108
  (1, 2090)	0.15497339408123192
  (1, 2440)	0.38396185486805534
  (1, 18170)	0.30092627119351906
  (1, 1659)	0.29508181443712256
  (1, 3052)	0.19146780561387253
  (1, 7437)	0.19085970824183243
  (1, 6015)	0.7131058243023835
  (1, 3864)	0.2632857340543818
  (2, 5880)	0.41085048965215354
  (2, 6526)	0.3648490352377015
  (2, 10403)	0.5182114550299733
  (2, 16892)	0.4405619707768525
  :	:
  (20797, 1444)	0.33544964410002415
  (20797, 14222)	0.24830376612959396
  (20797, 13402)	0.2726907293261386
  (20797, 16174)	0.3116587085589854
  (20797, 13188)	0.24783361133914847
  (20797, 10299)	

In [76]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,stratify=y,random_state=2)

In [77]:
model=LogisticRegression()

In [78]:
model.fit(x_train,y_train)
model.score(x_train,y_train)

0.9870192307692308

In [79]:
predections_XTrain=model.predict(x_train)
accuracy_score(y_train,predections_XTrain)

0.9870192307692308

In [80]:
predictions=model.predict(x_test)
accuracy_score(y_test,predictions)

0.9776442307692308

In [82]:
predictions=model.predict(x1)
ss=pd.read_csv('/content/submit.csv')
submission = ss.copy()
submission['label'] = predictions

# Save the submission file
submission.to_csv('submission.csv', index=False)